In [1]:
import pandas as pd
from mosaic_sql.postgres import get_engine as get_postgres_engine

In [2]:
postgres_engine = get_postgres_engine('ttda.postgres.storage.dev.mosaic.hartreepartners.com',
                                      '5437','postgres','p0stgresisforttda','postgres')

postgres_prod_engine = get_postgres_engine('ttda.postgres.storage.mosaic.hartreepartners.com',
                                      '5437','postgres','p0stgrespr0d4ttda','postgres')

In [3]:
df_dev = pd.read_sql("select * from settles.trader_curves where symbol like '%-TS' and date='2021-06-15'", con=postgres_engine)
df_prod = pd.read_sql("select * from settles.trader_curves where symbol like '%-TS' and date='2021-06-08'", con=postgres_prod_engine)

In [4]:
import json
df_dev['definition'] = df_dev['definition'].apply(lambda x: json.loads(x))
df_prod['definition'] = df_prod['definition'].apply(lambda x: json.loads(x))

In [21]:
df_dev['timing_factor'] = df_dev['definition'].apply(lambda x:x['timing_factor'])

TypeError: string indices must be integers

In [24]:
def split_dict(row):
    defn = json.loads(row['definition'])
    row['tf'] = defn['timing_factor']
    row['anchor_contract'] = defn['anchor_contract']
    row['marks'] = defn['marks']
    row['contracts'] = defn['contracts']
    return row
    


In [26]:
df_dev_n = df_dev.apply(split_dict, axis=1)
df_prod_n = df_prod.apply(split_dict, axis=1)

In [27]:
df_dev_n.to_csv(r'c:\temp\dev_ts.csv', index=False)
df_prod_n.to_csv(r'c:\temp\prod_ts.csv', index=False)